# Wetting Envelope Interactive Applet

This notebook provides an interactive interface for plotting wetting envelopes for materials and solvents. It allows you to:
- Plot wetting envelopes for materials with different surface energy components
- Add liquid points to the plot
- Adjust contact angles and other parameters

The calculations are based on the Owens-Wendt model for surface energy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import access_token
access_token.log_notebook_usage()

# Function to calculate R(phi) for the wetting envelope
def calculate_R(phi, sigma_s_p, sigma_s_d):
    cos_phi = np.cos(phi)
    sin_phi = np.sin(phi)
    R = (np.sqrt(cos_phi * sigma_s_d) + np.sqrt(sin_phi * sigma_s_p)) / (cos_phi + sin_phi)
    return R**2

# Function to plot the wetting envelope
def plot_wetting_envelope(components, thetas, legends, ax, title, correction_exp=2.0):
    ax.clear()
    plot_data = []
    
    # Plot each material's wetting envelope
    for i in range(len(thetas)):
        if len(components) > i:
            sigma_s_p = float(components[i][0])
            sigma_s_d = float(components[i][1])
        else:
            sigma_s_p = float(components[0][0])
            sigma_s_d = float(components[0][1])
            
        theta = float(thetas[i])
        
        # Convert theta to radians and calculate the correction factor
        theta_rad = np.radians(theta)
        correction_factor = 2 / (1 + np.cos(theta_rad))
        
        # Generate a range of phi values from 0 to 90 degrees (in radians) with 1000 points
        phi_values = np.linspace(0, np.pi / 2, 1000)
        R_values = calculate_R(phi_values, sigma_s_p, sigma_s_d)
        
        # Adjust R values by the correction factor
        R_values *= correction_factor**correction_exp
        
        # Convert to Cartesian coordinates where X is the dispersive component and Y is the polar component
        x_values = R_values * np.cos(phi_values)  # Dispersive component on X-axis
        y_values = R_values * np.sin(phi_values)  # Polar component on Y-axis
        
        # Store the data for export
        legend_label = legends[i] if i < len(legends) and legends[i] else f'θ = {theta}°'
        plot_data.append((x_values, y_values, legend_label))
        
        # Plotting the wetting envelope
        ax.plot(x_values, y_values, label=legend_label)
    
    # Plot liquid points
    for liquid in liquid_points:
        ax.scatter(liquid[1], liquid[0], label=liquid[2])
        plot_data.append(([liquid[1]], [liquid[0]], liquid[2]))
    
    if title:
        ax.set_title(title)
    ax.set_xlabel('Dispersive Component (mN/m)')
    ax.set_ylabel('Polar Component (mN/m)')
    ax.grid(True)
    ax.legend()
    
    # Determine axis limits with a margin
    if plot_data:
        x_max = max([max(x) for x, _, _ in plot_data if len(x) > 0]) * 1.1
        y_max = max([max(y) for _, y, _ in plot_data if len(y) > 0]) * 1.1
        ax.set_xlim([0, x_max])  # Ensure the X axis starts at 0 and has a margin
        ax.set_ylim([0, y_max])  # Ensure the Y axis starts at 0 and has a margin
    
    return plot_data

# Initialize global variable to store liquid points
liquid_points = []

In [2]:
# Default values for materials and solvents
# Default values for materials and solvents
default_materials = [
    {'name': 'Pure ITO', 'polar': 30.01, 'dispersive': 44.29},
    {'name': 'NiOx', 'polar': 29.74, 'dispersive': 46.41}
]

default_solvents = [
    {'name': 'Water', 'polar': 29.74, 'dispersive': 46.41},
    {'name': 'IPA/H2O', 'polar': 30.23, 'dispersive': 45.27}
                    ]

In [3]:
# Create widgets for the interactive interface
# Material input widgets
material_name = widgets.Text(value='Material 1', description='Name:')
material_polar = widgets.FloatText(value=5.43, description='Polar (mN/m):')
material_dispersive = widgets.FloatText(value=40.0, description='Dispersive (mN/m):')
material_theta = widgets.FloatSlider(value=0.0, min=0.0, max=180.0, step=1.0, description='Contact Angle (°):')

# Solvent input widgets
solvent_name = widgets.Text(value='Solvent 1', description='Name:')
solvent_polar = widgets.FloatText(value=18.0, description='Polar (mN/m):')
solvent_dispersive = widgets.FloatText(value=29.0, description='Dispersive (mN/m):')

# Plot settings widgets
plot_title = widgets.Text(value='Wetting Envelope', description='Title:')
correction_exponent = widgets.FloatSlider(value=2.0, min=1.0, max=2.0, step=0.01, description='Correction Exp:')

# Create buttons
add_material_button = widgets.Button(description='Add Material')
add_solvent_button = widgets.Button(description='Add Solvent')
clear_button = widgets.Button(description='Clear All')
plot_button = widgets.Button(description='Plot')

# Output area for the plot
output = widgets.Output()

# Lists to store materials and solvents
materials = []
solvents = []

# Add default materials and solvents
for material in default_materials:
    materials.append({
        'name': material['name'],
        'polar': material['polar'],
        'dispersive': material['dispersive'],
        'theta': 0.0
    })

for solvent in default_solvents:
    solvents.append({
        'name': solvent['name'],
        'polar': solvent['polar'],
        'dispersive': solvent['dispersive']
    })

# Display areas for materials and solvents
materials_output = widgets.Output()
solvents_output = widgets.Output()

In [4]:
# Define button callbacks
def add_material(b):
    materials.append({
        'name': material_name.value,
        'polar': material_polar.value,
        'dispersive': material_dispersive.value,
        'theta': material_theta.value
    })
    update_materials_display()

def add_solvent(b):
    solvents.append({
        'name': solvent_name.value,
        'polar': solvent_polar.value,
        'dispersive': solvent_dispersive.value
    })
    update_solvents_display()
    
def clear_all(b):
    global materials, solvents, liquid_points
    materials = []
    solvents = []
    liquid_points = []
    update_materials_display()
    update_solvents_display()
    with output:
        clear_output()

def plot_envelopes(b):
    global liquid_points
    with output:
        clear_output()
        
        # Prepare data for plotting
        components = [(str(m['polar']), str(m['dispersive'])) for m in materials]
        thetas = [str(m['theta']) for m in materials]
        legends = [m['name'] for m in materials]
        
        # Add solvents to liquid_points
        liquid_points = [(s['polar'], s['dispersive'], s['name']) for s in solvents]
        
        # Create figure and plot
        fig, ax = plt.subplots(figsize=(10, 8))
        plot_wetting_envelope(components, thetas, legends, ax, plot_title.value, correction_exponent.value)
        plt.tight_layout()
        plt.show()

# Update display functions
def update_materials_display():
    with materials_output:
        clear_output()
        if materials:
            print("Added Materials:")
            for i, m in enumerate(materials):
                print(f"{i+1}. {m['name']}: Polar={m['polar']}, Dispersive={m['dispersive']}, θ={m['theta']}°")
        else:
            print("No materials added.")

def update_solvents_display():
    with solvents_output:
        clear_output()
        if solvents:
            print("Added Solvents:")
            for i, s in enumerate(solvents):
                print(f"{i+1}. {s['name']}: Polar={s['polar']}, Dispersive={s['dispersive']}")
        else:
            print("No solvents added.")

# Connect callbacks to buttons
add_material_button.on_click(add_material)
add_solvent_button.on_click(add_solvent)
clear_button.on_click(clear_all)
plot_button.on_click(plot_envelopes)

# Initial display update
update_materials_display()
update_solvents_display()

In [5]:
# Create the layout for the interface
material_box = widgets.VBox([
    widgets.HTML(value="<h3>Material Properties</h3>"),
    material_name,
    material_polar,
    material_dispersive,
    material_theta,
    add_material_button,
    materials_output
])

solvent_box = widgets.VBox([
    widgets.HTML(value="<h3>Solvent Properties</h3>"),
    solvent_name,
    solvent_polar,
    solvent_dispersive,
    add_solvent_button,
    solvents_output
])

plot_settings_box = widgets.VBox([
    widgets.HTML(value="<h3>Plot Settings</h3>"),
    plot_title,
    correction_exponent,
    widgets.HBox([plot_button, clear_button])
])

# Arrange the interface
top_row = widgets.HBox([material_box, solvent_box])
app = widgets.VBox([top_row, plot_settings_box, output])

# Display the interface
display(app)

## How to Use This Applet

1. **Materials**: Enter the properties of materials (polar and dispersive components) and their contact angles.
2. **Solvents**: Enter the properties of solvents (polar and dispersive components).
3. **Plot Settings**: Adjust the plot title and correction exponent as needed.
4. **Buttons**:
   - **Add Material**: Add a material with the specified properties.
   - **Add Solvent**: Add a solvent with the specified properties.
   - **Plot**: Generate the wetting envelope plot.
   - **Clear All**: Remove all materials and solvents.

The plot shows wetting envelopes for each material and points for each solvent. If a solvent point falls inside a material's wetting envelope, it indicates that the solvent will wet the material.

In [6]:
# Add Voila configuration for deployment
# This cell is not necessary for running in Jupyter, but helps with Voila deployment
from IPython.display import HTML
HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide();
  });
</script>
<form action="javascript:code_toggle()">
<input type="submit" id="toggleButton" value="Show Code">
</form>
''')